# Reverting the BIOS to default values

<div class="alert alert-block alert-info">
<b>About: </b> here is brief abstract of the above example
</div>

In [1]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

<div class="alert alert-block alert-success">
<b> Brief of below funtion: </b> about below function
</div>

In [ ]:
def change_bios_setting(_redfishobj, bios_property, property_value, bios_password):

    bios_uri = None
    bios_data = None
    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the relevant URI
        
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
        bios_uri = systems_members_response.obj['Bios']['@odata.id']
        bios_data = _redfishobj.get(bios_uri)
    else:
        #Use Resource directory to find the relevant URI
        
        for instance in resource_instances:
            if '#Bios.' in instance['@odata.type']:
                bios_uri = instance['@odata.id']
                bios_data = _redfishobj.get(bios_uri)
                break

    if bios_data:
        print("\n\nShowing BIOS attributes before changes:\n\n")
        print(json.dumps(bios_data.dict, indent=4, sort_keys=True))

    if bios_uri:
        #BIOS settings URI is needed
        
        bios_settings_uri = bios_data.obj['@Redfish.Settings']['SettingsObject']['@odata.id']
        body = {'Attributes': {bios_property: property_value}}
        
        #update BIOS password
        if bios_password:
            _redfishobj.bios_password = bios_password
        resp = _redfishobj.patch(bios_settings_uri, body)

        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("\nSuccess!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))
            #uncomment if you would like to see the full list of attributes
            #print("\n\nShowing BIOS attributes after changes:\n\n")
            #bios_data = _redfishobj.get(bios_uri)
            #print(json.dumps(bios_data.dict, indent=4, sort_keys=True))


<div class="alert alert-block alert-success">
<b> Brief of below funtion: </b> about below function
</div>

In [ ]:
if __name__ == "__main__":
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD = input("Enter password: ")

    BIOS_PASSWORD = ""
    #provide the attribute name and the associated attribute value. Note: some, values may
    #be containers (arrays or dictionaries) so keep this in mind.
    ATTRIBUTE = "AdminName"
    ATTRIBUTE_VAL = "Luigi"
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, password=LOGIN_PASSWORD)
                                                                            
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    change_bios_setting(REDFISHOBJ, ATTRIBUTE, ATTRIBUTE_VAL, BIOS_PASSWORD)
    REDFISHOBJ.logout()

<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>